In [ ]:
result = pd.merge(routes, trips, on='route_id')
result = pd.merge(result, stopTimes, on='trip_id')
result = pd.merge(result, stops, on='stop_id')
result = result[result['route_id'] == 'OCE1506035']

#result['arrival_time'] = pd.to_datetime(result['arrival_time'])
#result['departure_time'] = pd.to_datetime(result['departure_time'])
#result['time'] = (result['departure_time'] - result['arrival_time'])
result[['route_id', 'stop_name']].groupby('route_id').head(5)


In [55]:
import pandas as pd
from datetime import datetime
import numpy as np
pd.set_option('display.expand_frame_repr', False)
from typing import List
import collections
import json
from copy import copy
from queue import PriorityQueue



In [ ]:
routes = pd.read_csv("../data/txt/routes.txt", usecols=['route_id', 'route_long_name'])
trips = pd.read_csv("../data/txt/trips.txt", usecols=['route_id', 'service_id', 'trip_id'])
stops = pd.read_csv("../data/txt/stops.txt", usecols=['stop_id', 'stop_name'])
stopTimes = pd.read_csv("../data/txt/stop_times.txt",usecols=['trip_id', 'stop_id', 'arrival_time', 'departure_time'])
calendar = pd.read_csv("../data/txt/calendar.txt")
calendar_date = pd.read_csv("../data/txt/calendar_dates.txt")
timetables = pd.read_csv('../data/csv/timetables.csv', sep='\t')

In [ ]:
timetables.head()

In [54]:
class Queue:
    def __init__(self):
        self.elements = collections.deque()
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, x):
        self.elements.append(x)
    
    def get(self):
        return self.elements.popleft()



In [127]:
class SimpleGraph:
    def __init__(self):
        self.edges: Dict[Location, Dict[string, float]] = {}
        timetables = pd.read_csv('../data/csv/timetables.csv', sep='\t')
        for index, row in timetables.iterrows():
            stopName = row['trajet'].split(' - ')
            time = row['duree']
            self.edges[stopName[0]] = {stopName[1]: time}

            if not stopName[1] in self.edges:
                self.edges[stopName[1]] = {}
    
    def neighbors(self, id ) -> List:
        return self.edges[id]
        
    def initVertex(self, start):
        inf = float('inf')
        self.vertex: Dict[string, Dict[float, string]] = {}
        for row in self.edges:
            self.vertex[row] = {'min': inf, 'from': ''}
        self.vertex[start]['min'] = 0


    def updateVertex(self, start):
        visited = Queue()
        unvisited = Queue()
        for row in self.vertex:
            unvisited.put(row)

        while not unvisited.empty():
            current = unvisited.get()
            for n in self.neighbors(current):
                actualCost = self.vertex[n]['min']
                newCost = self.vertex[current]['min'] + self.edges[current][n]
                if newCost < actualCost:
                    self.vertex[n]['min'] = min(actualCost, newCost)
                    self.vertex[n]['from'] = current
            visited.put(current)
    
    def getPath(self, start, end):
        self.initVertex(start)
        self.updateVertex(start)
        q = Queue()
        res = Queue()
        q.put(end)
        current = end
        print(self.vertex[end])
        while current != start:
            current = self.vertex[current]['from']
            q.put(current)
        
        while not q.empty():
            t = q.get()
            res.put(t)
        
        while not res.empty():
            print(res.get())
        print('opla')

graph = SimpleGraph()
graph.edges = {
    'A': {'B': 15, 'C': 11},
    'B': {'D': 5, 'E': 20},
    'C': {'D': 1},
    'E': {'B': 1},
    'D': {},
}
graph.getPath('A', 'D')

#graph.edges
#graph.edges['Gare de Serquigny']
#graph.getPath('Gare de Lyon-Part-Dieu', 'Gare de Valence-Ville')

{'min': 12, 'from': 'C'}
D
C
A
opla
